In [1]:
#RUN
from pandas import DataFrame
import pandas as pd
import numpy as np

In [2]:
#RUN
try:
    from urllib import urlencode
except ImportError:
    from urllib.parse import urlencode
    
from jsonapi_client import Session, Filter
import pycurl

API_BASE = 'https://www.ebi.ac.uk/metagenomics/api/latest/'

In [3]:
#RUN
# 'studies' bisa diganti dengan target semisal 'sample' ; 'MGYS00001939' merupakan accession
with Session(API_BASE) as s:
    std = s.get('studies', 'MGYS00003963').resource

In [4]:
#RUN
#memunculkan study_name
std

<ResourceObject: studies: MGYS00003963 (2174819581904) (2174819581904)>

In [5]:
#RUN
#melihat jumlah sample pada study di atas
std.samples_count

48

In [ ]:
#RUN/SKIP
sample = std.samples
#melihat informasi pada salah satu sample yang dipilih
x=sample[1]
x.sample_metadata

In [ ]:
def get_metadata(metadata, key):
    import html
    for m in metadata:
        if m['key'].lower() == key.lower():
            value = m['value']
            unit = html.unescape(m['unit']) if m['unit'] else ""
            return "{value} {unit}".format(value=value, unit=unit)
    return None

biome_label = 'environment (biome)'
elev_label = 'elevation'
df = DataFrame(columns=('Sample Name', 'Longitude', 'Latitude', 'Elevation', 'Biome', 'Country'))
df.index.name = 'ID Sample'

with Session(API_BASE) as s:
    params = {
        'study_accession': 'MGYS00003963',
        'page_size': 100,
    }
    f = Filter(urlencode(params))
    for sample in s.iterate('samples', f):
        for a in sample.runs:
        #print('runs: '+str(a.accession))
            for b in a.analyses:
                print('analyses: '+str(b.accession))
        df.loc[sample.accession] = [
            sample.sample_name,
            sample.longitude,
            sample.latitude,
            get_metadata(sample.sample_metadata, elev_label),
            get_metadata(sample.sample_metadata, biome_label),
            sample.geo_loc_name
        ]
df

In [7]:
def get_metadata(metadata, key):
    import html
    for m in metadata:
        if m['key'].lower() == key.lower():
            value = m['value']
            unit = html.unescape(m['unit']) if m['unit'] else ""
            return "{value} {unit}".format(value=value, unit=unit)
    return None

#biome_label = 'environment (biome)'
#elev_label = 'elevation'
#df = DataFrame(columns=('Sample Name', 'Longitude', 'Latitude', 'Elevation', 'Biome', 'Country'))
#df.index.name = 'ID Sample'
df = pd.DataFrame([])
acc = []
sample_name = []
analyze = []

with Session(API_BASE) as s:
    params = {
        'study_accession': 'MGYS00003963',
        'page_size': 100,
    }
    f = Filter(urlencode(params))
    for sample in s.iterate('samples', f):
        acc.append(sample.accession)
        sample_name.append(sample.sample_name)
        for a in sample.runs:
        #print('runs: '+str(a.accession))
            for b in a.analyses:
                print('analyses: '+str(b.accession))
        analyze.append(b.accession)
        meta = sample.sample_metadata
        meta = pd.DataFrame.from_dict(meta)
        meta = meta.set_index('key') 
        meta = meta.drop(columns='unit')
        meta = meta.transpose()
        df = df.append(meta)
acc = pd.DataFrame(acc, columns=['ID_Sample'])
sample_name = pd.DataFrame(sample_name, columns=['Sample Name'])
analyze = pd.DataFrame(analyze, columns=['ID_Analyses'])

analyses: MGYA00353758
analyses: MGYA00353759
analyses: MGYA00353760
analyses: MGYA00353761
analyses: MGYA00353762
analyses: MGYA00353763
analyses: MGYA00353764
analyses: MGYA00353765
analyses: MGYA00353766
analyses: MGYA00353767
analyses: MGYA00353768
analyses: MGYA00353769
analyses: MGYA00353770
analyses: MGYA00353771
analyses: MGYA00353772
analyses: MGYA00353773
analyses: MGYA00353774
analyses: MGYA00353775
analyses: MGYA00353776
analyses: MGYA00353777
analyses: MGYA00353778
analyses: MGYA00353779
analyses: MGYA00353780
analyses: MGYA00353781
analyses: MGYA00353782
analyses: MGYA00353783
analyses: MGYA00353784
analyses: MGYA00353785
analyses: MGYA00353786
analyses: MGYA00353787
analyses: MGYA00353788
analyses: MGYA00353789
analyses: MGYA00353790
analyses: MGYA00353791
analyses: MGYA00353792
analyses: MGYA00353793
analyses: MGYA00353794
analyses: MGYA00353795
analyses: MGYA00353796
analyses: MGYA00353797
analyses: MGYA00353798
analyses: MGYA00353799
analyses: MGYA00353800
analyses: M

In [ ]:
with Session(API_BASE) as s:
    params = {
        'study_accession': 'MGYS00003963',
        'page_size': 100,
    }
    f = Filter(urlencode(params))
    for sample in s.iterate('samples', f):
        for a in sample.runs:
        #print('runs: '+str(a.accession))
            for b in a.analyses:
                print('analyses: '+str(b.accession))

In [ ]:
#RUN
#memunculkan list sample sesuai jumlah
std.samples

In [ ]:
#SKIP
blank = []
for x in sample:
    #print(x.sample_metadata)
    df = pd.DataFrame(x.sample_metadata)
    df['key'].tolist()
    print(df)
    #df = pd.DataFrame(x.sample_metadata)
    #print(df)
    #index = np.where(df['key'].str.contains('location'))[0]
    #blank.append(df.iloc[index, :])
    #index = np.where(df['key'].str.contains('biome'))[0]
    #blank.append(df.iloc[index, :])
    #index = np.where(df['key'].str.contains('elevation'))[0]
    #blank.append(df.iloc[index, :])
    #df_new = pd.concat(blank)
#df_new

In [ ]:
#RUN
with Session(API_BASE) as s:
    std = s.get('studies', 'MGYS00003963').resource
print(std.accession)
#melihat informasi pada salah satu sample yang dipilih
sample = std.samples

In [ ]:
for x in sample:
    print('sample: '+str(x.accession))
    for a in x.runs:
        #print('runs: '+str(a.accession))
        for b in a.analyses:
            print('analyses: '+str(b.accession))

In [ ]:
#RUN
#membuat container kosong di bawah :
df = pd.DataFrame([])
acc = []
sample_name = []
analyze = []
for x in sample:
    acc.append(x.accession)
    sample_name.append(x.sample_name)
    for a in x.runs:
        #print('runs: '+str(a.accession))
        for b in a.analyses:
            print('analyses: '+str(b.accession))
    analyze.append(b.accession)
    meta = x.sample_metadata
    meta = pd.DataFrame.from_dict(meta)
    meta = meta.set_index('key') 
    meta = meta.drop(columns='unit')
    meta = meta.transpose()
    df = df.append(meta)
acc = pd.DataFrame(acc, columns=['ID_Sample'])
sample_name = pd.DataFrame(sample_name, columns=['Sample Name'])
analyze = pd.DataFrame(analyze, columns=['ID_Analyses'])

In [8]:
#RUN
#reset index agar dapat di merge
df = df.reset_index()
df = df.drop(columns='index')

In [9]:
#RUN
#script merge
merged = acc.merge(sample_name, left_index=True, right_index=True, how='inner')
merged = merged.merge(analyze, left_index=True, right_index=True, how='inner')
merged = merged.merge(df, left_index=True, right_index=True, how='inner')
merged

,ID_Sample,Sample Name,ID_Analyses,geographic location (longitude),"geographic location (country and/or sea,region)",environment (biome),environment (feature),environment (material),environmental package,depth,elevation,geographic location (latitude),instrument model,ENA checklist,last update date
0,ERS1474434,qiita_sid_846:846.Fagna24102011Soil12A1,MGYA00353758,11.342,Italy:Tuscany:Fagna,cropland biome,agricultural feature,agricultural soil,soil,0.0-0.1,224.0,43.984,Illumina HiSeq 2000,ERC000011,2016-12-16
1,ERS1474435,qiita_sid_846:846.Fagna24102011Soil12A2,MGYA00353759,11.342,Italy:Tuscany:Fagna,cropland biome,agricultural feature,agricultural soil,soil,0.1-0.2,224.0,43.984,Illumina HiSeq 2000,ERC000011,2016-12-16
2,ERS1474436,qiita_sid_846:846.Fagna24102011Soil12A3,MGYA00353760,11.342,Italy:Tuscany:Fagna,cropland biome,agricultural feature,agricultural soil,soil,0.2-0.3,224.0,43.984,Illumina HiSeq 2000,ERC000011,2016-12-16
3,ERS1474437,qiita_sid_846:846.Fagna24102011Soil12A4,MGYA00353761,11.342,Italy:Tuscany:Fagna,cropland biome,agricultural feature,agricultural soil,soil,0.3-0.4,224.0,43.984,Illumina HiSeq 2000,ERC000011,2016-12-16
4,ERS1474438,qiita_sid_846:846.Fagna24102011Soil12B1,MGYA00353762,11.342,Italy:Tuscany:Fagna,cropland biome,agricultural feature,agricultural soil,soil,0.0-0.1,224.0,43.984,Illumina HiSeq 2000,ERC000011,2016-12-16
5,ERS1474439,qiita_sid_846:846.Fagna24102011Soil12B2,MGYA00353763,11.342,Italy:Tuscany:Fagna,cropland biome,agricultural feature,agricultural soil,soil,0.1-0.2,224.0,43.984,Illumina HiSeq 2000,ERC000011,2016-12-16
6,ERS1474440,qiita_sid_846:846.Fagna24102011Soil12B3,MGYA00353764,11.342,Italy:Tuscany:Fagna,cropland biome,agricultural feature,agricultural soil,soil,0.2-0.3,224.0,43.984,Illumina HiSeq 2000,ERC000011,2016-12-16
7,ERS1474441,qiita_sid_846:846.Fagna24102011Soil12B4,MGYA00353765,11.342,Italy:Tuscany:Fagna,cropland biome,agricultural feature,agricultural soil,soil,0.3-0.4,224.0,43.984,Illumina HiSeq 2000,ERC000011,2016-12-16
8,ERS1474442,qiita_sid_846:846.Fagna24102011Soil12C1,MGYA00353766,11.341,Italy:Tuscany:Fagna,cropland biome,agricultural feature,agricultural soil,soil,0.0-0.1,224.0,43.984,Illumina HiSeq 2000,ERC000011,2016-12-16
9,ERS1474443,qiita_sid_846:846.Fagna24102011Soil12C2,MGYA00353767,11.341,Italy:Tuscany:Fagna,cropland biome,agricultural feature,agricultural soil,soil,0.1-0.2,224.0,43.984,Illumina HiSeq 2000,ERC000011,2016-12-16


In [ ]:
#RUN SEKALI AJA
#membuat folder
from pathlib import Path
import os
_dir = os.getcwd()
#_dir = 'D:\\Seminar\\Seminar_Anam'
_dir = os.path.join(_dir, 'Study_Jagung')
if not os.path.exists(_dir):
    os.makedirs(_dir)
print(_dir)

In [ ]:
#RUN
#masuk ke folder "Study_Jagung"
%cd Study_Jagung
#export ke .csv
merged.to_csv(std.accession+'.csv')

In [ ]:
#SKIP
for x in sample:
    #print('sample: '+str(i.accession))
    for a in x.runs:
        #print('runs: '+str(a.accession))
        for b in a.analyses:
            print('analyses: '+str(b.accession))

In [ ]:
#SKIP
#memasukkan sample yang dipilih ke bentuk kolom
df = pd.DataFrame(sample[0].sample_metadata)
df

In [ ]:
#SKIP
blank = []

In [ ]:
#SKIP
#membuat index baru dari list 'key' yang dipilih
index = np.where(df['key'].str.contains('location'))[0]
blank.append(df.iloc[index, :])
index = np.where(df['key'].str.contains('biome'))[0]
blank.append(df.iloc[index, :])
index = np.where(df['key'].str.contains('elevation'))[0]
blank.append(df.iloc[index, :])

In [ ]:
#SKIP
#'key' yang dilih dijadikan data frame baru
df_new = pd.concat(blank)
df_new

In [ ]:
#SKIP
filename_sample = '' 

In [ ]:
#SKIP
#membuat loop accession di dalam bentuk list
for x in sample:
    #print('sample: '+str(i.accession))
    for a in x.runs:
        #print('runs: '+str(a.accession))
        for b in a.analyses:
            print('analyses: '+str(b.accession))
            #for z in y.downloads:
                #if 'json.biom' in z.url:
                    #print(z.url)

In [ ]:
#SKIP
#membuat folder
from pathlib import Path
import os
_dir = os.getcwd()
#_dir = 'D:\\Seminar\\Seminar_Anam'
_dir = os.path.join(_dir, 'Study_Jagung')
if not os.path.exists(_dir):
    os.makedirs(_dir)
print(_dir)
#%rmdir(Study+str)
#with open('_dir'+i.accession+'.biom', 'wb') as f:
    #c = pycurl.Curl()
    #c.setopt(c.URL, z.url)
    #c.setopt(c.WRITEDATA, f)
    #c.perform()
#c.close()